In [2]:
import sys
import torch
import numpy as np
sys.path.append('..')
from pathlib import Path
from typing import Union
from ml_utilities.torch_models.base_model import BaseModel
from ml_utilities.torch_models import get_model_class
from omegaconf import OmegaConf
from erank.regularization import EffectiveRankRegularization
import matplotlib.pyplot as plt

/system/apps/userenv/beck/erank/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def erank_np(matrix_A: np.ndarray):
    s = np.linalg.svd(matrix_A, compute_uv=False)
    probs = s / s.sum()
    entropy = (- probs * np.log(probs)).sum()
    return np.exp(entropy)

In [4]:
def erank_torch(matrix_A: torch.Tensor, center_matrix_A: bool=False) -> torch.Tensor:
    """Calculates the effective rank of a matrix.
    Args:
        matrix_A (torch.Tensor): Matrix of shape m x n. 
        center_matrix_A (bool): Center the matrix 
    Returns:
        torch.Tensor: Effective rank of matrix_A
    """
    assert matrix_A.ndim == 2
    _, s, _ = torch.pca_lowrank(matrix_A, center=center_matrix_A, niter=1, q=min(matrix_A.shape[0], matrix_A.shape[1]))
    # normalizes input s -> scale independent!
    return torch.exp(torch.distributions.Categorical(s).entropy())

In [5]:
matrix_A = np.random.normal(size=(2,3))
matrix_A

array([[-1.37023893e+00,  1.00492783e+00,  1.51135325e+00],
       [ 8.84128539e-04,  2.45242460e-01, -2.86567595e-01]])

In [6]:
erank_np(matrix_A), erank_torch(torch.from_numpy(matrix_A))

(1.496870115192069, tensor(1.1259, dtype=torch.float64))